In [11]:
from pyproj import Proj, transform

# 定义原始地理坐标系和目标投影坐标系
# WGS84是一个常用的地理坐标系
wgs84 = Proj(init='epsg:4326')
# 假设我们要转换到UTM坐标系，WGS 84 / UTM zone 33N
utm = Proj(init='epsg:32633')

# 示例经纬度坐标
longitude = 109.879
latitude = 33.592

# 转换坐标
x, y = transform(wgs84, utm, longitude, latitude)
print(f"投影坐标系下的坐标: X={x}, Y={y}")

投影坐标系下的坐标: X=8069317.015811941, Y=10806710.763501817


E:\Anaconda3\envs\map\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
E:\Anaconda3\envs\map\lib\site-packages\pyproj\crs\crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
E:\Anaconda3\envs\map\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  


In [1]:
import pandas as pd
import numpy as np

# 读取TFW文件内容
A = 0.0000001616
B = 0.0000000000
C = 0.0000000000
D = -0.0000001352
E = 109.5142514878
F = 33.6084263172

# tif栅格图像的尺寸
image_width = 17410
image_height = 17954

# 读取Excel文件，指定engine为openpyxl
df = pd.read_excel("output_excel_path.xlsx", engine='openpyxl')

# 定义函数来计算中心点和尺寸
def calculate_center_and_size(points):
    longitudes = [point[0] for point in points]
    latitudes = [point[1] for point in points]
    
    # 计算中心点
    center_longitude = np.mean(longitudes)
    center_latitude = np.mean(latitudes)
    
    # 计算宽度和高度（单位：像素）
    pixel_coords = [(int((lon - E) / A), int((lat - F) / D)) for lon, lat in points]
    xs, ys = zip(*pixel_coords)
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    
    return center_longitude, center_latitude, width, height

# 初始化YOLO标签列表
yolo_labels = []

# 遍历每一行数据
for index, row in df.iterrows():
    points = [
        eval(row[1]),  # 读取第2列的点
        eval(row[2]),  # 读取第3列的点
        eval(row[3]),  # 读取第4列的点
        eval(row[4])   # 读取第5列的点
    ]
    
    # 计算中心点和尺寸
    center_longitude, center_latitude, object_width_in_pixels, object_height_in_pixels = calculate_center_and_size(points)
    
    # 将地理坐标转换为图像像素坐标
    pixel_x = (center_longitude - E) / A
    pixel_y = (center_latitude - F) / D
    
    # 计算YOLO标签中的相对坐标
    center_x = pixel_x / image_width
    center_y = pixel_y / image_height
    width = object_width_in_pixels / image_width
    height = object_height_in_pixels / image_height
    
    # 类别ID（假设为0）
    class_id = 0
    
    # 生成YOLO标签
    yolo_label = f"{class_id} {center_x} {center_y} {width} {height}"
    
    # 添加到YOLO标签列表
    yolo_labels.append(yolo_label)

# 打印结果
for label in yolo_labels:
    print("YOLO标签：", label)

# 可选：将标签写入文件
with open("yolo_label_path.txt", "w") as label_file:
    for label in yolo_labels:
        label_file.write(label + "\n")

YOLO标签： 0 0.02538462763441275 0.34871297696619896 0.005226881102814474 0.006516653670491255
